# Выбор локации для скважины

Допустим, мы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построим модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import seaborn as sns
import warnings
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats as st

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Доход за тысячу бареллей
INCOME = 450000
# Число необходимых скважин
WELLS = 200
# значение random_state
RS = 12345
# Бюджет 
BUDGET = 10_000_000_000

In [2]:
display(HTML("<style>.container { width:70% !important; <style>}"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")

In [3]:
try:
    df_1 = pd.read_csv('E:/BuduBogatym/geo_data_0.csv')
    df_2 = pd.read_csv('E:/BuduBogatym/geo_data_1.csv')
    df_3 = pd.read_csv('E:/BuduBogatym/geo_data_2.csv')
except:
    df_1 = pd.read_csv(
        'https://code.s3.yandex.net/datasets/geo_data_0.csv')
    df_2 = pd.read_csv(
        'https://code.s3.yandex.net/datasets/geo_data_1.csv')
    df_3 = pd.read_csv(
        'https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [4]:
df_1.info()
df_2.info()
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

Во всех датасетах отсутствуют пропущенные значения, а также корректны типы данных в столбцах.

In [5]:
print('Дубликатов в 1 датасете',df_1['id'].duplicated().sum())
print('Дубликатов в 2 датасете',df_2['id'].duplicated().sum())
print('Дубликатов в 3 датасете',df_3['id'].duplicated().sum())


Дубликатов в 1 датасете 10
Дубликатов в 2 датасете 4
Дубликатов в 3 датасете 4


Так как дубликатов не так много, то я предлагаю избавиться от них

In [6]:
# избавимся от дубликатов
df_1.drop_duplicates(subset = 'id',inplace=True)
df_2.drop_duplicates(subset = 'id',inplace=True)
df_3.drop_duplicates(subset = 'id',inplace=True)


In [7]:
print('Дубликатов в 1 датасете',df_1['id'].duplicated().sum())
print('Дубликатов в 2 датасете',df_2['id'].duplicated().sum())
print('Дубликатов в 3 датасете',df_3['id'].duplicated().sum())

Дубликатов в 1 датасете 0
Дубликатов в 2 датасете 0
Дубликатов в 3 датасете 0


## Обучение и проверка модели

In [8]:
features_df1 = df_1.drop(['product','id'], axis=1)
features_df2 = df_2.drop(['product','id'], axis=1)
features_df3 = df_3.drop(['product','id'], axis=1)
target_df1 = df_1['product']
target_df2 = df_2['product']
target_df3 = df_3['product']

In [9]:
def split_data(features,target):
    train_features, valid_features, train_target, valid_target = train_test_split(
        features, target, test_size=0.25, random_state = RS)
    return train_features, valid_features, train_target, valid_target

### Обучение модели на 1 датасете

In [10]:
# Обучим модель линейной регрессии на 1 датасете
%time
train_features_df1, valid_features_df1, train_target_df1, valid_target_df1 = split_data(features_df1, target_df1)

# Проверим размерность выборок
print(train_features_df1.shape)
print(valid_features_df1.shape)
print(train_target_df1.shape)
print(valid_target_df1.shape)

model_df1 = LinearRegression()
model_df1.fit(train_features_df1,train_target_df1)
predictions_valid_df1 = model_df1.predict(valid_features_df1)

CPU times: total: 0 ns
Wall time: 0 ns
(74992, 3)
(24998, 3)
(74992,)
(24998,)


Рассчитаем RMSE и средний запас предсказанного сырья

In [11]:
# Посчитаем средний запас предсказанного сырья
mean_df1 = predictions_valid_df1.mean()
# посчитаем значение метрики RMSE на валидационной выборке
rmse_df1 = mean_squared_error(valid_target_df1,predictions_valid_df1)**0.5# посчитайте значение метрики RMSE на валидационной выборке

print("RMSE модели линейной регрессии на валидационной выборке первой модели:", rmse_df1)
print('Средний запас предсказанного сырье на 1 модели', mean_df1)

RMSE модели линейной регрессии на валидационной выборке первой модели: 37.853527328872964
Средний запас предсказанного сырье на 1 модели 92.78915638280621


### Обучение модели на 2 датасете

In [12]:
# Обучим модель линейной регрессии на 2 датасете
%time
train_features_df2, valid_features_df2, train_target_df2, valid_target_df2 = split_data(features_df2, target_df2)

# Проверим размерность выборок
print(train_features_df2.shape)
print(valid_features_df2.shape)
print(train_target_df2.shape)
print(valid_target_df2.shape)

model_df2 = LinearRegression()
model_df2.fit(train_features_df2,train_target_df2)
predictions_valid_df2 = model_df2.predict(valid_features_df2)

CPU times: total: 0 ns
Wall time: 0 ns
(74997, 3)
(24999, 3)
(74997,)
(24999,)


Рассчитаем RMSE и средний запас предсказанного сырья

In [13]:
# Посчитаем средний запас предсказанного сырья
mean_df2 = predictions_valid_df2.mean()
# посчитаем значение метрики RMSE на валидационной выборке
rmse_df2 = mean_squared_error(valid_target_df2,predictions_valid_df2)**0.5

print("RMSE модели линейной регрессии на валидационной выборке второй модели:", rmse_df2)
print('Средний запас предсказанного сырье на второй модели', mean_df2)

RMSE модели линейной регрессии на валидационной выборке второй модели: 0.8920592647717028
Средний запас предсказанного сырье на второй модели 69.1783195703043


### Обучение модели на 3 датасете

In [14]:
# Обучим модель линейной регрессии на 3 датасете
%time
train_features_df3, valid_features_df3, train_target_df3, valid_target_df3 = split_data(features_df3, target_df3)

# Проверим размерность выборок
print(train_features_df3.shape)
print(valid_features_df3.shape)
print(train_target_df3.shape)
print(valid_target_df3.shape)

model_df3 = LinearRegression()
model_df3.fit(train_features_df3,train_target_df3)
predictions_valid_df3 = model_df3.predict(valid_features_df3)

CPU times: total: 0 ns
Wall time: 0 ns
(74997, 3)
(24999, 3)
(74997,)
(24999,)


Рассчитаем RMSE и средний запас предсказанного сырья

In [15]:
# Посчитаем средний запас предсказанного сырья
mean_df3 = predictions_valid_df3.mean()
# посчитаем значение метрики RMSE на валидационной выборке
rmse_df3 = mean_squared_error(valid_target_df3,predictions_valid_df3)**0.5

print("RMSE модели линейной регрессии на валидационной выборке третьей модели:", rmse_df3)
print('Средний запас предсказанного сырье на третьей модели', mean_df3)

RMSE модели линейной регрессии на валидационной выборке третьей модели: 40.07585073246016
Средний запас предсказанного сырье на третьей модели 94.86572480562035


Согласно полученному результату можно сделать выбор в пользу модели по второму датасету, так как используемая нами метрика RMSE имеет отличный показатель чуть меньше 1.
Однако остальные две модели имеют больший средний запас, но это не так важно, так как RMSE отражает сумму квадратов ошибок, а показатели этих моделей по метрике RMSE крайне плохие.

## Подготовка к расчёту прибыли

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины и сравним эти значения со средним запасом в каждом регионе

In [16]:
# Расчёт объёма сырья для безубыточной разработки новой скважины
stock = BUDGET / (WELLS * INCOME)
print('Достаточный объём сырья для безубыточной разработки новой скважины:', stock)
# Сравнение необходимых значений друг с другом
for i in [df_1, df_2, df_3]:
    print(stock > i['product'].mean())

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111
True
True
True


По полученным результатам можно сделать вывод о том, что больше половины скавжин в каждом регионе не будет подходить для нашей задачи

## Расчёт прибыли и рисков 

In [17]:
predictions_valid_df1 = pd.Series(predictions_valid_df1)
predictions_valid_df2 = pd.Series(predictions_valid_df2)
predictions_valid_df3 = pd.Series(predictions_valid_df3)

In [18]:
# Функция расчёта прибыли по выбранным скважинам
def revenue(target, predictions, count):
    best_index = predictions.sort_values(ascending=False)[
        :count].index.values
    selected = target.iloc[best_index]
    return INCOME * selected.sum() - BUDGET

In [19]:
print('Прибыль первого региона:', round((revenue(valid_target_df1, predictions_valid_df1, 200)/10**9),2), 'млрд. руб.')

Прибыль первого региона: 3.37 млрд. руб.


In [20]:
print('Прибыль второго региона:', round((revenue(valid_target_df2, predictions_valid_df2, 200)/10**9),2), 'млрд. руб.')

Прибыль второго региона: 2.42 млрд. руб.


In [21]:
print('Прибыль третьего региона:', round((revenue(valid_target_df3, predictions_valid_df3, 200)/10**9),2), 'млрд. руб.')

Прибыль третьего региона: 2.5 млрд. руб.


Далее с помощью бустрепа рассчитаем доверительный интервал, среднюю выручку и риск убытков.

### Расчёт первого региона

In [22]:
state = np.random.RandomState(RS)

valid_target_df1.reset_index(drop = 'first')

values = []
# Бустреп модели 
for i in range(1000):
    predict_subsample = predictions_valid_df1.sample(n=500, replace=True, random_state=state)
    predict_subsample = pd.Series(predict_subsample.sort_values(ascending=False)[:WELLS].index.values)
    target_subsample = valid_target_df1.iloc[predict_subsample] 

    values.append(revenue(target_subsample, predict_subsample, WELLS))

In [23]:
# Расчёт необходимых параметров
values = pd.Series(values)
mean = values.mean()

confidence_interval = st.t.interval(0.95 , values.count()-1 ,values.mean() , values.sem())
risk = (values < 0).mean()
 

print('Доверительный интервал', confidence_interval)
print("Средняя выручка:", mean)
print("Риск убытков:", risk*100,'%')

Доверительный интервал (364560620.98509455, 396666319.05010056)
Средняя выручка: 380613470.01759756
Риск убытков: 7.7 %


### Расчёт второго региона

In [24]:
valid_target_df2.reset_index(drop = 'first')

values = []
 
for i in range(1000):
    predict_subsample = predictions_valid_df2.sample(n=500, replace=True, random_state=state)
    predict_subsample = pd.Series(predict_subsample.sort_values(ascending=False)[:WELLS].index.values)
    target_subsample = valid_target_df2.iloc[predict_subsample] 

    values.append(revenue(target_subsample, predict_subsample, WELLS))

In [25]:
values = pd.Series(values)

mean = values.mean()

confidence_interval = st.t.interval(0.95 , values.count()-1 ,values.mean() , values.sem())
risk = (values < 0).mean()
 

print('Доверительный интервал', confidence_interval)
print("Средняя выручка:", mean)
print("Риск убытков:", risk*100,'%')

Доверительный интервал (466404387.9390931, 491915352.0240818)
Средняя выручка: 479159869.98158747
Риск убытков: 0.6 %


### Расчёт третьего региона

In [26]:
valid_target_df3.reset_index(drop = 'first')

values = []
 
for i in range(1000):
    predict_subsample = predictions_valid_df3.sample(n=500, replace=True, random_state=state)
    predict_subsample = pd.Series(predict_subsample.sort_values(ascending=False)[:WELLS].index.values)
    target_subsample = valid_target_df3.iloc[predict_subsample] 

    values.append(revenue(target_subsample, predict_subsample, WELLS))

In [27]:
values = pd.Series(values)

mean = values.mean()

confidence_interval = st.t.interval(0.95 , values.count()-1 ,values.mean() , values.sem())
risk = (values < 0).mean()
 

print('Доверительный интервал', confidence_interval)
print("Средняя выручка:", mean)
print("Риск убытков:", risk*100,'%')

Доверительный интервал (298798348.57535076, 332085483.471629)
Средняя выручка: 315441916.0234899
Риск убытков: 12.4 %


## Вывод:
---
   * В ходе работы на основе трёх регионов были построены модели и произведена оценка метрики RMSE, согласно которой лучшим показателем обладает второй регион.
   ---
   * Разработана функция определения прибыли для каждого региона, результат использования которой представлен ниже:
       * Прибыль первого региона: 3.37 млрд. руб.
       * Прибыль второго региона: 2.42 млрд. руб.
       * Прибыль третьего региона: 2.5 млрд. руб.
       ---
   * Произведён расчёт рисков по каждому региону со следующими результатами:
       * Риск убытков первого региона: 7.7 %
       * Риск убытков второго региона: 0.6 %
       * Риск убытков третьего региона: 12.4 %
       ---
По итогам исследования можно дать рекомендации о выборе второго региона, не смотря на то, что прибыль его не самая высокая, но риск убытков вписывается в необходимые 2,5%